# Random Forest from scratch!

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from fastai.imports import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics

In [3]:
PATH = "data/bulldozers/"

## Load in our data from last lesson

In [ ]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False,
                     converters={ 'SalePrice': lambda sp: np.log(float(sp)) },
                     parse_dates=["saledate"])

In [ ]:
add_datepart(df_raw, 'saledate')

train_cats(df_raw)
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)
df_raw.UsageBand = df_raw.UsageBand.cat.codes

In [ ]:
df_raw.to_feather(f'{PATH}raw')

In [4]:
df_raw = pd.read_feather(f'{PATH}raw')

In [5]:
# df_raw = pd.read_feather('tmp/raw')
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')

In [6]:
def split_vals(a,n): return a[:n], a[n:]
n_valid = 12000
n_trn = len(df_trn)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
raw_train, raw_valid = split_vals(df_raw, n_trn)

In [ ]:
x_sub = X_train[['YearMade', 'MachineHoursCurrentMeter']]

## Basic data structures

In [ ]:
# All randomness is handled in this class

class TreeEnsemble():
    def __init__(self, x, y, n_trees, sample_sz, min_leaf=5):
        np.random.seed(42)  # explicitly set for reproducibility
        self.x,self.y,self.sample_sz,self.min_leaf = x,y,sample_sz,min_leaf
        self.trees = [self.create_tree() for i in range(n_trees)]

    def create_tree(self):
        # construct decision tree from random sample of data
        rnd_idxs = np.random.permutation(len(self.y))[:self.sample_sz]  # not bootstrapping (sampling w/ replacement)
        return DecisionTree(self.x.iloc[rnd_idxs], self.y[rnd_idxs], min_leaf=self.min_leaf)
        
    def predict(self, x):
        # for given row(s), average the predictions generated by all of the trees
        return np.mean([t.predict(x) for t in self.trees], axis=0)

In [ ]:
# DecisionTrees are recursive so we pass along all the arguments above (except n_trees)
class DecisionTree():
    def __init__(self, x, y, idxs=None, min_leaf=5):
        # need to keep track of which idxs went to left/right side of the tree
        # initialize it to all dependent variables
        if idxs is None: idxs=np.arange(len(y))
        self.x,self.y,self.idxs,self.min_leaf = x,y,idxs,min_leaf
        # n => rows
        # c => columns
        self.n,self.c = len(idxs), x.shape[1]
        # initial value is the average of dependent variable for the idxs
        self.val = np.mean(y[idxs])
        # score: effectiveness of the split  (not for leaf nodes; root of the tree - no splits yet)
        self.score = float('inf')
        self.find_varsplit()
        
    # This just does one decision; we'll make it recursive later
    def find_varsplit(self):
        # NOTE: we are not including max_features support.  It would go here.
        for i in range(self.c): self.find_better_split(i)
            
    # We'll write this later!
    def find_better_split(self, var_idx): pass
    
    @property
    def split_name(self): return self.x.columns[self.var_idx]
    
    @property
    def split_col(self): return self.x.values[self.idxs,self.var_idx]

    # method call w/out the ()
    @property     # python decorator -> tells python info about your method
    def is_leaf(self): return self.score == float('inf')
    
    # representation of the object -> information returned when printed (instead of classname and objectID)
    def __repr__(self):
        # rows, avg of dependent variable 
        s = f'n: {self.n}; val:{self.val}'
        if not self.is_leaf:
            # if there is a split: show score, value split at, variable split on
            s += f'; score:{self.score}; split:{self.split}; var:{self.split_name}'
        return s

In [ ]:
m = TreeEnsemble(x_sub, y_train, n_trees=1, sample_sz=1000, min_leaf=3)

In [ ]:
tree = m.trees[0]; tree

In [ ]:
m.trees[0].idxs

In [ ]:
x_samp,y_samp = tree.x, tree.y
x_samp.columns

## Single split

### Find best split given variable

In [ ]:
# get information from scikit_learn implementation so we can compare
# we need to recreate this information...

m = RandomForestRegressor(n_estimators=1, max_depth=1, bootstrap=False)
m.fit(x_samp, y_samp)
draw_tree(m.estimators_[0], x_samp, precision=2)

In [ ]:
def find_better_split(self, var_idx):
    # x: array of independent variable values
    # y: dependent variable
    x,y = self.x.values[self.idxs,var_idx], self.y[self.idxs]

    # want to find a split where each side has as low a std as possible
    for i in range(1,self.n-1):
        # split into 2 groups (left hand side, right hand side)
        lhs = x<=x[i]     # array of booleans
        rhs = x>x[i]
        if rhs.sum()==0: continue   # can't take std of empty set -> sum() counts True values
        # get std of both groups
        lhs_std = y[lhs].std()
        rhs_std = y[rhs].std()
        # take weighted average of the 2 groups; keep the lowest one
        curr_score = lhs_std*lhs.sum() + rhs_std*rhs.sum()
        if curr_score<self.score: 
            self.var_idx,self.score,self.split = var_idx,curr_score,x[i]

In [ ]:
%timeit find_better_split(tree,1)
tree

In [ ]:
# this is the better score -> split here
# compare w/ sklearn and it matches up correctly!
find_better_split(tree,0); tree

#### Calculating Computational Complexity

- Is there a loop?  -> n  
- Is there a loop inside the loop?  -> n * n  
- Is there a sort?  -> n(log(n))  
- Element-wise array operations  -> n  

complexity of loop x highest complexity inside loop  
n x n (element-wise array operation)  ->  n^2

#### Increase performance  ->  Decrease computation complexity

calculate aggregated std -> square root of (mean of squares/n) - (square of means/n)  
- Sort the values
- RHS - Calculate total counts, sum of values, sum of values squared
- For each value we can add to LHS / subtract from RHS

order of n complexity

### Speeding things up

In [ ]:
tree = TreeEnsemble(x_sub, y_train, 1, 1000).trees[0]

In [ ]:
# s1: sum of values
# s2: sum of values^2
# square root of (mean of squares/n) - (square of means/n)
def std_agg(cnt, s1, s2): return math.sqrt((s2/cnt) - (s1/cnt)**2)

def find_better_split(self, var_idx):
    x,y = self.x.values[self.idxs,var_idx], self.y[self.idxs]
    
    sort_idx = np.argsort(x)
    sort_y,sort_x = y[sort_idx], x[sort_idx]
    # calc initial values -> everything starts in rhs, none in lhs
    # counts, values, values^2
    rhs_cnt,rhs_sum,rhs_sum2 = self.n, sort_y.sum(), (sort_y**2).sum()
    lhs_cnt,lhs_sum,lhs_sum2 = 0,0.,0.

    for i in range(0,self.n-self.min_leaf-1):
        xi,yi = sort_x[i],sort_y[i]
        # update counts, values, and values^2
        lhs_cnt += 1; rhs_cnt -= 1
        lhs_sum += yi; rhs_sum -= yi
        lhs_sum2 += yi**2; rhs_sum2 -= yi**2
        # if this value is same as next one -> skip rest
        if i<self.min_leaf or xi==sort_x[i+1]: continue
            
        # calculate std
        lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2)
        rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2)
        curr_score = lhs_std*lhs_cnt + rhs_std*rhs_cnt
        if curr_score<self.score: 
            self.var_idx,self.score,self.split = var_idx,curr_score,xi

In [ ]:
# 800% performance improvement...

%timeit find_better_split(tree,1)
tree

In [ ]:
find_better_split(tree,0); tree

In [ ]:
DecisionTree.find_better_split = find_better_split

In [ ]:
tree = TreeEnsemble(x_sub, y_train, 1, 1000).trees[0]; tree

## Full single tree

In [ ]:
# now lets go deeper -> max_depth=2 -> 2 splits
m = RandomForestRegressor(n_estimators=1, max_depth=2, bootstrap=False)
m.fit(x_samp, y_samp)
draw_tree(m.estimators_[0], x_samp, precision=2)

In [ ]:
# We now need to make this recursive
def find_varsplit(self):
    # same as above
    for i in range(self.c): self.find_better_split(i)
    # breaks our circular recursion
    if self.is_leaf: return
    # property -> returns x values for this column
    x = self.split_col
    # storing booleans for the entire array for each node -> squared memory issue
    # lets store just the indexes
    lhs = np.nonzero(x<=self.split)[0]  # np.nonzero -> gives the indexes of the true values
    rhs = np.nonzero(x>self.split)[0]
    # create decision trees for both sides
    self.lhs = DecisionTree(self.x, self.y, self.idxs[lhs])
    self.rhs = DecisionTree(self.x, self.y, self.idxs[rhs])

In [ ]:
DecisionTree.find_varsplit = find_varsplit

In [ ]:
tree = TreeEnsemble(x_sub, y_train, 1, 1000).trees[0]; tree

In [ ]:
tree.lhs

In [ ]:
tree.rhs

In [ ]:
tree.lhs.lhs

In [ ]:
tree.lhs.rhs

## Predictions

In [7]:
cols = ['MachineID', 'YearMade', 'MachineHoursCurrentMeter', 'ProductSize', 'Enclosure',
        'Coupler_System', 'saleYear']

In [ ]:
%time tree = TreeEnsemble(X_train[cols], y_train, 1, 1000).trees[0]
x_samp,y_samp = tree.x, tree.y

In [ ]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False)
m.fit(x_samp, y_samp)
draw_tree(m.estimators_[0], x_samp, precision=2, ratio=0.9, size=7)

In [ ]:
# predictions for a tree are the predictions for a row for every row

# loops through leading axis regardless of the rank (num of axes)
# 1 - vector -> vector
# 2 - matrix -> rows
# 3 - tensor -> matrices which represent slices

def predict(self, x): return np.array([self.predict_row(xi) for xi in x])
DecisionTree.predict = predict

In [ ]:
if something:
    x= do1()
else:
    x= do2()

In [ ]:
x = do1() if something else do2()

In [ ]:
x = something ? do1() : do2()

In [ ]:
# This is also recursive
def predict_row(self, xi):
    # if at a leaf node -> return the value
    if self.is_leaf: return self.val 
    # if not at a leaf node -> need to decide to go down left or right to get prediction
    # lhs if less than split otherwise rhs
    t = self.lhs if xi[self.var_idx]<=self.split else self.rhs
    # call predict_row again
    return t.predict_row(xi)

DecisionTree.predict_row = predict_row

In [ ]:
%time preds = tree.predict(X_valid[cols].values)

In [ ]:
plt.scatter(preds, y_valid, alpha=0.05)
# alpha -> transparency
# 20 dots will appear fully blue

In [ ]:
metrics.r2_score(preds, y_valid)

In [ ]:
# compare to sklearn RF

m = RandomForestRegressor(n_estimators=1, min_samples_leaf=5, bootstrap=False)
%time m.fit(x_samp, y_samp)
preds = m.predict(X_valid[cols].values)
plt.scatter(preds, y_valid, alpha=0.05)

In [ ]:
metrics.r2_score(preds, y_valid)

# Putting it together

In [8]:
cols = ['MachineID', 'YearMade', 'MachineHoursCurrentMeter', 'ProductSize', 'Enclosure',
        'Coupler_System', 'saleYear']

In [45]:
class TreeEnsemble():
    def __init__(self, x, y, n_trees, sample_sz, min_leaf=5):
        np.random.seed(42)
        self.x,self.y,self.sample_sz,self.min_leaf = x,y,sample_sz,min_leaf
        self.trees = [self.create_tree() for i in range(n_trees)]

    def create_tree(self):
        idxs = np.random.permutation(len(self.y))[:self.sample_sz]
        return DecisionTree(self.x.iloc[idxs], self.y[idxs], 
                    idxs=np.array(range(self.sample_sz)), min_leaf=self.min_leaf)
        
    def predict(self, x):
        return np.mean([t.predict(x) for t in self.trees], axis=0)

def std_agg(cnt, s1, s2): return math.sqrt((s2/cnt) - (s1/cnt)**2)

In [46]:
class DecisionTree():
    def __init__(self, x, y, idxs, min_leaf=5):
        self.x,self.y,self.idxs,self.min_leaf = x,y,idxs,min_leaf
        self.n,self.c = len(idxs), x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
        
    def find_varsplit(self):
        for i in range(self.c): self.find_better_split(i)
        if self.score == float('inf'): return
        x = self.split_col
        lhs = np.nonzero(x<=self.split)[0]
        rhs = np.nonzero(x>self.split)[0]
        self.lhs = DecisionTree(self.x, self.y, self.idxs[lhs])
        self.rhs = DecisionTree(self.x, self.y, self.idxs[rhs])

    def find_better_split(self, var_idx):
        x,y = self.x.values[self.idxs,var_idx], self.y[self.idxs]
        sort_idx = np.argsort(x)
        sort_y,sort_x = y[sort_idx], x[sort_idx]
        rhs_cnt,rhs_sum,rhs_sum2 = self.n, sort_y.sum(), (sort_y**2).sum()
        lhs_cnt,lhs_sum,lhs_sum2 = 0,0.,0.

        for i in range(0,self.n-self.min_leaf-1):
            xi,yi = sort_x[i],sort_y[i]
            lhs_cnt += 1; rhs_cnt -= 1
            lhs_sum += yi; rhs_sum -= yi
            lhs_sum2 += yi**2; rhs_sum2 -= yi**2
            if i<self.min_leaf or xi==sort_x[i+1]:
                continue

            lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2)
            rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2)
            curr_score = lhs_std*lhs_cnt + rhs_std*rhs_cnt
            if curr_score<self.score: 
                self.var_idx,self.score,self.split = var_idx,curr_score,xi

    @property
    def split_name(self): return self.x.columns[self.var_idx]
    
    @property
    def split_col(self): return self.x.values[self.idxs,self.var_idx]

    @property
    def is_leaf(self): return self.score == float('inf')
    
#     def __repr__(self):
#         s = f'n: {self.n}; val:{self.val}'
#         if not self.is_leaf:
#             s += f'; score:{self.score}; split:{self.split}; var:{self.split_name}'
#         return s

    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        t = self.lhs if xi[self.var_idx]<=self.split else self.rhs
        return t.predict_row(xi)

In [11]:
ens = TreeEnsemble(X_train[cols], y_train, 5, 1000)

In [40]:
preds = ens.predict(X_valid[cols].values)

In [ ]:
plt.scatter(y_valid, preds, alpha=0.1, s=6);

In [41]:
metrics.r2_score(y_valid, preds)

0.71011741571071241

# Improve Performance

This is a lot slower than sklearn 
- many of calculations are numpy (optimized)
- other calcs are in python (leaf nodes calculating score) -> slow

In [ ]:
%load_ext Cython

In [ ]:
def fib1(n):
    a, b = 0, 1
    while b < n:
        a, b = b, a + b

In [ ]:
%%cython
def fib2(n):
    a, b = 0, 1
    while b < n:
        a, b = b, a + b

In [ ]:
%%cython
def fib3(int n):
    # cdef: defining c data types
    cdef int b = 1
    cdef int a = 0
    # need another variable here because we can't batch define variables in c
    cdef int t = 0
    while b < n:
        t = a
        a = b
        b = a + b

In [ ]:
%timeit fib1(50)

In [ ]:
# 2 times faster
%timeit fib2(50)

In [ ]:
# 10 times faster
%timeit fib3(50)

# Confidence (tree variance) 

In [26]:
x_samp,y_samp = ens.trees[0].x, ens.trees[0].y

m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False)
m.fit(X_train[cols], y_train)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [39]:
m_preds = np.stack([t.predict(X_valid[cols]) for t in m.estimators_])
np.mean(m_preds[:,0]), np.std(m_preds[:,0])

(10.008139878485583, 0.0)

In [50]:
??m

In [ ]:
%time preds = np.stack([t.predict(X_valid) for t in m.estimators_])
np.mean(preds[:,0]), np.std(preds[:,0])

In [ ]:
def get_preds(t): return t.predict(X_valid)
preds = np.stack(parallel_trees(ens, get_preds))
np.mean(preds[:,0]), np.std(preds[:,0])

# Feature Importance

variable shuffling

# Partial Dependence

instead of shuffling -> replacing w/ constant value

# Tree Interpreter

## Interaction Importance

alt feature importance: adding up all rows